In [6]:
import requests
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.data.dataset import random_split


In [7]:
def get_multiverse_ids(set_code):
    url = f"https://api.scryfall.com/cards/search?order=set&q=e:{set_code}&unique=prints"
    response = requests.get(url)
    
    if response.status_code != 200:
        raise Exception(f"Error fetching data from Scryfall: {response.status_code}")
    
    data = response.json()
    multiverse_ids = []
    
    while True:
        for card in data['data']:
            if 'multiverse_ids' in card:
                multiverse_ids.extend(card['multiverse_ids'])
        
        if data['has_more']:
            next_page = data['next_page']
            response = requests.get(next_page)
            if response.status_code != 200:
                raise Exception(f"Error fetching next page from Scryfall: {response.status_code}")
            data = response.json()
        else:
            break
    
    return multiverse_ids

def download_card_images(multiverse_ids, folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    downloaded_count = 0
    skipped_count = 0
    for mid in multiverse_ids:
        image_path = os.path.join(folder_path, f"{mid}.jpg")
        if os.path.exists(image_path):
            skipped_count += 1
            continue  # Skip downloading if the image already exists
        
        image_url = f"https://gatherer.wizards.com/Handlers/Image.ashx?multiverseid={mid}&type=card"
        try:
            response = requests.get(image_url)
            if response.status_code == 200:
                with open(image_path, 'wb') as file:
                    file.write(response.content)
                downloaded_count += 1
            else:
                print(f"Failed to download image for multiverse ID {mid}: HTTP {response.status_code}")
        except Exception as e:
            print(f"Error downloading image for multiverse ID {mid}: {e}")
    
    return f"Downloaded {downloaded_count} images, skipped {skipped_count} existing images."

class ImageDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.image_paths = [os.path.join(directory, fname) for fname in os.listdir(directory) if fname.lower().endswith(('png', 'jpg', 'jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = F.one_hot(torch.tensor(idx), num_classes=len(self.image_paths)).float()
        return image, label

def create_dataloader(directory, batch_size=32, shuffle=True):
    transform = transforms.Compose([
        transforms.Resize((265, 370)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
    ])
    
    dataset = ImageDataset(directory=directory, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

def train_model(train_dataloader, test_dataloader, model, epochs, learning_rate=0.001):
    torch.cuda.empty_cache()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        for inputs, labels in train_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Test the model with test dataset after each epoch
        model.eval()  # Set model to evaluation mode
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                predicted = outputs.argmax(dim=1)
                total += labels.size(0)
                correct += (predicted == labels.argmax(dim=1)).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_dataloader)}, Test Accuracy: {accuracy}%")

    print('Finished Training')
    
def split_dataloader(dataloader, train_ratio=0.8):
    # Get the original dataset from the DataLoader
    dataset = dataloader.dataset
    
    # Calculate the sizes for training and testing splits
    train_size = int(len(dataset) * train_ratio)
    test_size = len(dataset) - train_size
    
    # Randomly split the dataset into training and testing datasets
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    
    # Create new DataLoaders for the training and testing datasets
    # It's important to keep the batch size and other parameters consistent
    train_dataloader = DataLoader(train_dataset, batch_size=dataloader.batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=dataloader.batch_size, shuffle=False)
    
    return train_dataloader, test_dataloader

In [8]:
class VarCNN(nn.Module):
    def __init__(self, num_classes):
        super(VarCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=64*16*23, out_features=8192)
        self.fc2 = nn.Linear(in_features=8192, out_features=1024)
        self.fc3 = nn.Linear(in_features=1024, out_features=num_classes)

    def forward(self, x):
        # Define the forward pass
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        x = x.view(-1, 64 * 16 * 23)  # Adjust the size here based on your input image size
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [9]:
set_code = "SPG"  # Replace with your desired set code
multiverse_ids = get_multiverse_ids(set_code)
print(f"Found {len(multiverse_ids)} cards in set {set_code}")
print(download_card_images(multiverse_ids, set_code))
train_loader = create_dataloader(set_code)
test_loader = create_dataloader(set_code)
model = VarCNN(num_classes=len(train_loader.dataset.image_paths))

Found 48 cards in set SPG
Downloaded 0 images, skipped 48 existing images.


In [10]:
train_model(train_loader, test_loader, model, epochs=1000)

Epoch 1/1000, Loss: 5.38494348526001, Test Accuracy: 2.0833333333333335%


OutOfMemoryError: CUDA out of memory. Tried to allocate 736.00 MiB. GPU 0 has a total capacty of 8.00 GiB of which 0 bytes is free. Of the allocated memory 6.10 GiB is allocated by PyTorch, and 820.06 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF